## import

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import quote
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()

#keyword = '최강배달꾼'

In [2]:
import pandas as pd

## 데이터

In [3]:
title_list = pd.read_csv('./title_list.csv')

In [4]:
title_list = title_list[['index', 'asset_nm_new']]

In [5]:
title_list = title_list[title_list['index'] <= 241]

In [6]:
print(len(title_list))
print(type(title_list))

242
<class 'pandas.core.frame.DataFrame'>


In [7]:
def web_data(keyword):
    
    driver = webdriver.Chrome()
    
    # Watcha 검색 페이지로 이동
    search_url = f'https://pedia.watcha.com/ko-KR/search?query={keyword}'
    driver.get(search_url)
    
    print('진행중 : ', keyword)

    # 검색 결과 중에서 특정 부분 클릭 (예: 첫 번째 검색 결과 클릭)
    try:
        main_xpath = '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div/div[1]/div/ul/li/a/div[1]/div[1]/img'
        driver.find_element(By.XPATH, main_xpath).click()

        # 페이지가 로딩될 때까지 잠시 기다림 (시간은 상황에 따라 조절)
        time.sleep(2)

        # 더보기 클릭
        more_xpath = '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[2]/section[2]/header/div/div/a'
        driver.get(driver.find_element(By.XPATH, more_xpath).get_attribute("href"))

        # 페이지가 로딩될 때까지 잠시 기다림 (시간은 상황에 따라 조절)
        time.sleep(2)

        # 리뷰 부분의 요소를 찾습니다.
        review_xpath = '//*[@id="root"]/div/div[1]/section/section/div/div/div/ul/div[1]/div[2]/a/div/span'
        review = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, review_xpath)))

        # 스크롤 다운을 통해 추가 리뷰를 로드
        pagedowns = 10
        while pagedowns:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.1)
            pagedowns -= 1

        html = BeautifulSoup(driver.page_source, 'html.parser')
        #print(html)

        # 리뷰를 담고 있는 요소들을 찾기
        review_elements = html.select('#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section')

        # 각 리뷰의 텍스트를 저장
        output_list = []
        for review_element in review_elements:
            for child in range(1, 31):
                try:
                    review_text = review_element.select_one('#root > div > div.css-126e3ta-NavContainer.ed8qmey1 > section > section > div > div > div > ul > div:nth-child(' + str(child) + ') > div.css-2occzs.egj9y8a1 > a > div > span').get_text(strip=True)
                    #print(f'Review: {review_text}')
                    output_list.append(review_text + '###!!!')
                except:
                    pass
    
    # 리뷰가 없거나 검색 결과가 없는 등의 예외 상황이 발생할 때의 처리      
    except:
        output_list = ['No']
        pass

    # 창 닫기
    driver.quit()

    # 모은 리뷰를 텍스트 파일로 저장
    output_file = open('./review files/' + keyword + '.txt', 'w', encoding = 'utf8')
    for item in output_list:
        output_file.write(item)

    # 파일 close
    output_file.close()



In [ ]:
# 제목에 특수 기호가 들어가면 파일 이름으로 사용하지 못하므로 이를 대체해서 저장하고 후에 복구
title_list['asset_nm_new'] = title_list['asset_nm_new'].str.replace('?', '...')

In [15]:
# 데이터 크롤링 수행
title_data = title_list[236:]
for _, title_ in title_data.iterrows():
    web_data(title_['asset_nm_new'])

진행중 :  거미입니다만문제라도...
진행중 :  범죄도시2
진행중 :  체리툰
진행중 :  베이비버스인기동요시즌4
진행중 :  먹놀잠프로젝트
진행중 :  봉신방


In [14]:
title_list[title_list['index'] == 236]

,index,asset_nm_new
236,236,거미입니다만문제라도...
